# Custom Exception Handling

---

## Table of Contents
1. [Introduction](#introduction)
2. [Built-in Exception Hierarchy](#exception-hierarchy)
3. [Creating Custom Exceptions](#creating-custom-exceptions)
4. [Exception with Additional Data](#exception-with-data)
5. [Exception Chaining](#exception-chaining)
6. [Exception Hierarchy Design](#hierarchy-design)
7. [Custom Exception Best Practices](#best-practices)
8. [Real-World Examples](#real-world-examples)
9. [Advanced Custom Exceptions](#advanced-exceptions)
10. [When to Create Custom Exceptions](#when-to-create)
11. [Testing Custom Exceptions](#testing-exceptions)
12. [Summary](#summary)

---

## 1. Introduction <a id='introduction'></a>

**Custom Exceptions** are user-defined exception classes that extend Python's built-in exception hierarchy to handle application-specific errors.

**Key Points:**
- Custom exceptions make error handling more specific and meaningful
- They provide better error messages and context
- Help distinguish between different types of errors in your application
- Make code more maintainable and easier to debug

**Why Create Custom Exceptions?**
1. **Specificity**: More precise error information
2. **Clarity**: Self-documenting error types
3. **Control**: Better error handling flow
4. **Context**: Include application-specific information

**Real-Life Analogy:**
Think of custom exceptions like specific warning lights in a car. Instead of just a generic "error" light, you have specific indicators for engine problems, low oil, brake issues, etc. Each one tells you exactly what's wrong and what action to take.

---

## 2. Built-in Exception Hierarchy <a id='exception-hierarchy'></a>

Understanding Python's built-in exception hierarchy helps you create well-designed custom exceptions.

### Python Exception Hierarchy:

```
BaseException
 ├── SystemExit
 ├── KeyboardInterrupt
 └── Exception
      ├── ArithmeticError
      │    ├── ZeroDivisionError
      │    ├── OverflowError
      │    └── FloatingPointError
      ├── LookupError
      │    ├── IndexError
      │    └── KeyError
      ├── ValueError
      ├── TypeError
      ├── AttributeError
      ├── NameError
      └── ... (many more)
```

### Common Built-in Exceptions:

| Exception | Description |
|-----------|-------------|
| `Exception` | Base class for all exceptions (except system-exiting) |
| `ValueError` | Raised when a value is inappropriate |
| `TypeError` | Raised when an operation is applied to wrong type |
| `AttributeError` | Raised when attribute reference or assignment fails |
| `KeyError` | Raised when a dictionary key is not found |
| `IndexError` | Raised when sequence index is out of range |
| `FileNotFoundError` | Raised when a file is not found |
| `RuntimeError` | Raised when error doesn't fit other categories |

**Important:** Always inherit from `Exception` or its subclasses, not `BaseException`.

---

## 3. Creating Custom Exceptions <a id='creating-custom-exceptions'></a>

Custom exceptions are created by inheriting from the `Exception` class or one of its subclasses.

### Basic Custom Exception:

```python
class CustomError(Exception):
    pass
```

### Why This Works:
- Inherits all exception behavior from `Exception`
- Can be caught specifically
- Provides meaningful error type name

In [ ]:
# Example 1: Simple Custom Exception

class InvalidAgeError(Exception):
    """Raised when age is invalid"""
    pass

def set_age(age):
    """Set age with validation"""
    if age < 0:
        raise InvalidAgeError("Age cannot be negative")
    if age > 150:
        raise InvalidAgeError("Age cannot be greater than 150")
    return age

# Test the custom exception
try:
    print("Setting age to 25:")
    age = set_age(25)
    print(f"Age set to: {age}")
    
    print("\nSetting age to -5:")
    age = set_age(-5)
    print(f"Age set to: {age}")
except InvalidAgeError as e:
    print(f"Error: {e}")
    print(f"Exception type: {type(e).__name__}")

In [ ]:
# Example 2: Multiple Custom Exceptions

class InsufficientFundsError(Exception):
    """Raised when account has insufficient funds"""
    pass

class NegativeAmountError(Exception):
    """Raised when amount is negative"""
    pass

class AccountClosedError(Exception):
    """Raised when account is closed"""
    pass

class BankAccount:
    """Simple bank account with custom exception handling"""
    
    def __init__(self, balance=0):
        self.balance = balance
        self.is_closed = False
    
    def withdraw(self, amount):
        """Withdraw money from account"""
        if self.is_closed:
            raise AccountClosedError("Cannot withdraw from closed account")
        
        if amount < 0:
            raise NegativeAmountError("Withdrawal amount cannot be negative")
        
        if amount > self.balance:
            raise InsufficientFundsError(
                f"Insufficient funds. Balance: ${self.balance:.2f}, Requested: ${amount:.2f}"
            )
        
        self.balance -= amount
        return self.balance
    
    def close(self):
        """Close the account"""
        self.is_closed = True

# Test different custom exceptions
account = BankAccount(100)

print("Test 1: Valid withdrawal")
try:
    new_balance = account.withdraw(30)
    print(f"Success! New balance: ${new_balance:.2f}\n")
except Exception as e:
    print(f"Error: {e}\n")

print("Test 2: Insufficient funds")
try:
    account.withdraw(200)
except InsufficientFundsError as e:
    print(f"InsufficientFundsError: {e}\n")

print("Test 3: Negative amount")
try:
    account.withdraw(-50)
except NegativeAmountError as e:
    print(f"NegativeAmountError: {e}\n")

print("Test 4: Closed account")
account.close()
try:
    account.withdraw(10)
except AccountClosedError as e:
    print(f"AccountClosedError: {e}")

---

## 4. Exception with Additional Data <a id='exception-with-data'></a>

Custom exceptions can store additional data beyond just an error message.

### Adding Attributes:

You can add attributes to provide more context about the error:
- Error codes
- Timestamp
- User information
- Suggested fixes
- Related data

In [ ]:
# Example: Custom Exception with Additional Data

class ValidationError(Exception):
    """Exception for validation errors with detailed information"""
    
    def __init__(self, message, field_name, invalid_value, valid_range=None):
        super().__init__(message)
        self.field_name = field_name
        self.invalid_value = invalid_value
        self.valid_range = valid_range
    
    def __str__(self):
        """Custom string representation"""
        msg = f"ValidationError in field '{self.field_name}': {self.args[0]}"
        msg += f"\n  Invalid value: {self.invalid_value}"
        if self.valid_range:
            msg += f"\n  Valid range: {self.valid_range}"
        return msg

def validate_temperature(temp):
    """Validate temperature reading"""
    if temp < -273.15:
        raise ValidationError(
            "Temperature cannot be below absolute zero",
            field_name="temperature",
            invalid_value=temp,
            valid_range=">= -273.15°C"
        )
    if temp > 1000:
        raise ValidationError(
            "Temperature reading seems unrealistic",
            field_name="temperature",
            invalid_value=temp,
            valid_range="<= 1000°C"
        )
    return temp

# Test with invalid temperature
try:
    validate_temperature(-300)
except ValidationError as e:
    print(e)
    print(f"\nAccessing exception attributes:")
    print(f"  Field: {e.field_name}")
    print(f"  Invalid value: {e.invalid_value}")
    print(f"  Valid range: {e.valid_range}")

In [ ]:
# Example: Custom Exception with Error Code

class APIError(Exception):
    """Exception for API errors with error codes"""
    
    def __init__(self, message, error_code, status_code=None):
        super().__init__(message)
        self.error_code = error_code
        self.status_code = status_code
    
    def __str__(self):
        msg = f"[Error {self.error_code}] {self.args[0]}"
        if self.status_code:
            msg = f"HTTP {self.status_code}: {msg}"
        return msg

class AuthenticationError(APIError):
    """Raised when authentication fails"""
    
    def __init__(self, message):
        super().__init__(message, error_code="AUTH001", status_code=401)

class RateLimitError(APIError):
    """Raised when rate limit is exceeded"""
    
    def __init__(self, message, retry_after=None):
        super().__init__(message, error_code="RATE001", status_code=429)
        self.retry_after = retry_after
    
    def __str__(self):
        msg = super().__str__()
        if self.retry_after:
            msg += f" (Retry after {self.retry_after} seconds)"
        return msg

# Test API errors
print("Test 1: Authentication Error")
try:
    raise AuthenticationError("Invalid API key")
except APIError as e:
    print(e)
    print(f"Error code: {e.error_code}\n")

print("Test 2: Rate Limit Error")
try:
    raise RateLimitError("Too many requests", retry_after=60)
except RateLimitError as e:
    print(e)
    print(f"Retry after: {e.retry_after} seconds")

---

## 5. Exception Chaining <a id='exception-chaining'></a>

**Exception chaining** allows you to preserve the original exception when raising a new one.

### Two Methods:

1. **Explicit chaining** with `from`: `raise NewException from original_exception`
2. **Implicit chaining**: Automatic when exception occurs during exception handling

### Benefits:
- Preserves debugging information
- Shows complete error chain
- Helps trace root cause

In [ ]:
# Example: Exception Chaining

class DatabaseError(Exception):
    """Raised when database operation fails"""
    pass

class ConfigurationError(Exception):
    """Raised when configuration is invalid"""
    pass

def connect_to_database(config):
    """Simulate database connection"""
    try:
        # Simulate connection error
        if 'host' not in config:
            raise KeyError("'host' key not found in configuration")
        # Simulate connection
        raise ConnectionError("Could not connect to database server")
    except KeyError as e:
        # Explicit chaining with 'from'
        raise ConfigurationError(
            "Invalid database configuration"
        ) from e
    except ConnectionError as e:
        # Explicit chaining
        raise DatabaseError(
            "Failed to establish database connection"
        ) from e

# Test exception chaining
print("Test 1: Missing configuration key")
try:
    connect_to_database({})
except ConfigurationError as e:
    print(f"Caught: {e}")
    print(f"Caused by: {e.__cause__}")
    print(f"Original type: {type(e.__cause__).__name__}\n")

print("Test 2: Connection failure")
try:
    connect_to_database({'host': 'localhost'})
except DatabaseError as e:
    print(f"Caught: {e}")
    print(f"Caused by: {e.__cause__}")

---

## 6. Exception Hierarchy Design <a id='hierarchy-design'></a>

Designing a good exception hierarchy helps organize related exceptions and enables flexible error handling.

### Benefits of Exception Hierarchies:

1. **Catch by category**: Catch all related exceptions with base class
2. **Specific handling**: Catch specific exceptions when needed
3. **Organization**: Logical grouping of related errors
4. **Extensibility**: Easy to add new exception types

In [ ]:
# Example: Custom Exception Hierarchy

# Base exception for the application
class PaymentError(Exception):
    """Base exception for payment-related errors"""
    pass

# Specific payment errors
class PaymentDeclinedError(PaymentError):
    """Raised when payment is declined"""
    
    def __init__(self, message, reason=None):
        super().__init__(message)
        self.reason = reason

class InsufficientFundsError(PaymentDeclinedError):
    """Raised when card has insufficient funds"""
    
    def __init__(self, message, available_funds, required_funds):
        super().__init__(message, reason="insufficient_funds")
        self.available_funds = available_funds
        self.required_funds = required_funds

class InvalidCardError(PaymentDeclinedError):
    """Raised when card is invalid"""
    
    def __init__(self, message, card_number):
        super().__init__(message, reason="invalid_card")
        self.card_number = card_number[-4:]  # Store only last 4 digits

class PaymentProcessingError(PaymentError):
    """Raised when payment processing fails"""
    pass

class NetworkError(PaymentProcessingError):
    """Raised when network connection fails"""
    pass

class PaymentGatewayError(PaymentProcessingError):
    """Raised when payment gateway fails"""
    
    def __init__(self, message, gateway_name):
        super().__init__(message)
        self.gateway_name = gateway_name

# Simulate payment processing
def process_payment(amount, card_number, scenario="success"):
    """Simulate payment processing with different scenarios"""
    
    if scenario == "insufficient_funds":
        raise InsufficientFundsError(
            "Card has insufficient funds",
            available_funds=50.00,
            required_funds=amount
        )
    elif scenario == "invalid_card":
        raise InvalidCardError(
            "Card number is invalid",
            card_number=card_number
        )
    elif scenario == "network_error":
        raise NetworkError("Network connection lost")
    elif scenario == "gateway_error":
        raise PaymentGatewayError(
            "Payment gateway timeout",
            gateway_name="Stripe"
        )
    
    return f"Payment of ${amount:.2f} processed successfully"

# Test exception hierarchy
test_cases = [
    ("success", "Success case"),
    ("insufficient_funds", "Insufficient funds"),
    ("invalid_card", "Invalid card"),
    ("network_error", "Network error"),
    ("gateway_error", "Gateway error")
]

for scenario, description in test_cases:
    print(f"\nTest: {description}")
    try:
        result = process_payment(100.00, "1234-5678-9012-3456", scenario)
        print(f"  Result: {result}")
    
    # Catch specific exceptions
    except InsufficientFundsError as e:
        print(f"  InsufficientFundsError: {e}")
        print(f"  Available: ${e.available_funds:.2f}, Required: ${e.required_funds:.2f}")
    
    except InvalidCardError as e:
        print(f"  InvalidCardError: {e}")
        print(f"  Card ending in: {e.card_number}")
    
    except NetworkError as e:
        print(f"  NetworkError: {e}")
    
    except PaymentGatewayError as e:
        print(f"  PaymentGatewayError: {e}")
        print(f"  Gateway: {e.gateway_name}")
    
    # Catch all payment errors
    except PaymentError as e:
        print(f"  PaymentError (generic): {e}")

---

## 7. Custom Exception Best Practices <a id='best-practices'></a>

### 1. Naming Conventions
- Use descriptive names ending in "Error" or "Exception"
- Example: `ValueError`, `DatabaseError`, `AuthenticationError`

### 2. Inherit from Appropriate Base Class
- Inherit from `Exception` or its subclasses
- Never inherit from `BaseException` directly
- Consider inheriting from more specific exceptions (e.g., `ValueError`)

### 3. Add Docstrings
- Document when the exception is raised
- Explain what it means
- Include usage examples

### 4. Include Useful Information
- Provide context in error messages
- Add relevant attributes
- Suggest fixes when possible

### 5. Keep Exception Classes Simple
- Don't add complex logic to exceptions
- Focus on data and representation

### 6. Use Exception Chaining
- Preserve original exception with `from`
- Helps with debugging

### 7. Create Exception Hierarchies
- Group related exceptions under base classes
- Enables flexible error handling

### 8. Don't Overuse Custom Exceptions
- Use built-in exceptions when appropriate
- Only create custom exceptions when they add value

In [ ]:
# Best Practice Example: Well-designed Custom Exception

class UserInputError(ValueError):
    """
    Raised when user input is invalid.
    
    This exception is raised when user-provided data fails validation.
    It includes the field name, invalid value, and suggested fix.
    
    Attributes:
        field_name (str): Name of the field with invalid input
        invalid_value: The value that failed validation
        suggestion (str): Suggested fix for the user
    
    Example:
        >>> raise UserInputError(
        ...     "Invalid email format",
        ...     field_name="email",
        ...     invalid_value="notanemail",
        ...     suggestion="Email must contain @ symbol"
        ... )
    """
    
    def __init__(self, message, field_name, invalid_value, suggestion=None):
        # Call parent constructor
        super().__init__(message)
        
        # Store additional data
        self.field_name = field_name
        self.invalid_value = invalid_value
        self.suggestion = suggestion
    
    def __str__(self):
        """Provide detailed error message"""
        msg = f"Invalid input for '{self.field_name}': {self.args[0]}"
        msg += f"\n  Received: {self.invalid_value}"
        if self.suggestion:
            msg += f"\n  Suggestion: {self.suggestion}"
        return msg
    
    def __repr__(self):
        """Unambiguous representation for debugging"""
        return (f"{self.__class__.__name__}("
                f"message={self.args[0]!r}, "
                f"field_name={self.field_name!r}, "
                f"invalid_value={self.invalid_value!r}, "
                f"suggestion={self.suggestion!r})")

# Test well-designed exception
try:
    raise UserInputError(
        "Email format is invalid",
        field_name="email",
        invalid_value="notanemail",
        suggestion="Email must be in format: user@domain.com"
    )
except UserInputError as e:
    print("Error occurred:")
    print(e)
    print("\nRepr for debugging:")
    print(repr(e))

---

## 8. Real-World Examples <a id='real-world-examples'></a>

Let's explore comprehensive real-world examples of custom exception systems.

In [ ]:
# Example 1: File Processing System with Custom Exceptions

class FileProcessingError(Exception):
    """Base exception for file processing errors"""
    
    def __init__(self, message, filename):
        super().__init__(message)
        self.filename = filename

class FileFormatError(FileProcessingError):
    """Raised when file format is invalid"""
    
    def __init__(self, message, filename, expected_format, actual_format=None):
        super().__init__(message, filename)
        self.expected_format = expected_format
        self.actual_format = actual_format

class FileSizeError(FileProcessingError):
    """Raised when file size exceeds limits"""
    
    def __init__(self, message, filename, file_size, max_size):
        super().__init__(message, filename)
        self.file_size = file_size
        self.max_size = max_size

class FileContentError(FileProcessingError):
    """Raised when file content is invalid"""
    
    def __init__(self, message, filename, line_number=None):
        super().__init__(message, filename)
        self.line_number = line_number

class FileProcessor:
    """File processor with comprehensive error handling"""
    
    MAX_FILE_SIZE = 10 * 1024 * 1024  # 10 MB
    
    def process_file(self, filename, file_size, file_extension, content_valid=True):
        """Process a file with validation"""
        
        # Check file format
        if file_extension not in ['.txt', '.csv', '.json']:
            raise FileFormatError(
                f"Unsupported file format",
                filename=filename,
                expected_format=".txt, .csv, or .json",
                actual_format=file_extension
            )
        
        # Check file size
        if file_size > self.MAX_FILE_SIZE:
            raise FileSizeError(
                f"File size exceeds maximum allowed size",
                filename=filename,
                file_size=file_size,
                max_size=self.MAX_FILE_SIZE
            )
        
        # Check content validity
        if not content_valid:
            raise FileContentError(
                f"File contains invalid data",
                filename=filename,
                line_number=42
            )
        
        return f"Successfully processed {filename}"

# Test file processor
processor = FileProcessor()

test_cases = [
    ("data.txt", 1024, ".txt", True, "Valid file"),
    ("data.pdf", 1024, ".pdf", True, "Invalid format"),
    ("huge.txt", 20 * 1024 * 1024, ".txt", True, "File too large"),
    ("corrupt.csv", 1024, ".csv", False, "Invalid content")
]

for filename, size, ext, valid, description in test_cases:
    print(f"\nTest: {description}")
    try:
        result = processor.process_file(filename, size, ext, valid)
        print(f"  Success: {result}")
    
    except FileFormatError as e:
        print(f"  FileFormatError: {e.args[0]}")
        print(f"  File: {e.filename}")
        print(f"  Expected: {e.expected_format}, Got: {e.actual_format}")
    
    except FileSizeError as e:
        print(f"  FileSizeError: {e.args[0]}")
        print(f"  File: {e.filename}")
        print(f"  Size: {e.file_size / (1024*1024):.1f} MB (max: {e.max_size / (1024*1024):.1f} MB)")
    
    except FileContentError as e:
        print(f"  FileContentError: {e.args[0]}")
        print(f"  File: {e.filename}")
        if e.line_number:
            print(f"  Error at line: {e.line_number}")
    
    except FileProcessingError as e:
        print(f"  FileProcessingError (generic): {e}")

In [ ]:
# Example 2: E-commerce System with Custom Exceptions

import datetime

class OrderError(Exception):
    """Base exception for order-related errors"""
    
    def __init__(self, message, order_id=None):
        super().__init__(message)
        self.order_id = order_id
        self.timestamp = datetime.datetime.now()

class OutOfStockError(OrderError):
    """Raised when product is out of stock"""
    
    def __init__(self, message, order_id, product_name, requested_qty, available_qty):
        super().__init__(message, order_id)
        self.product_name = product_name
        self.requested_qty = requested_qty
        self.available_qty = available_qty
    
    def __str__(self):
        return (f"{self.args[0]} for '{self.product_name}': "
                f"Requested {self.requested_qty}, Available {self.available_qty}")

class InvalidDiscountCodeError(OrderError):
    """Raised when discount code is invalid"""
    
    def __init__(self, message, order_id, discount_code, reason=None):
        super().__init__(message, order_id)
        self.discount_code = discount_code
        self.reason = reason

class ShippingError(OrderError):
    """Raised when shipping address is invalid"""
    
    def __init__(self, message, order_id, address_field):
        super().__init__(message, order_id)
        self.address_field = address_field

class Order:
    """Simplified order class"""
    
    def __init__(self, order_id):
        self.order_id = order_id
        self.items = {}
    
    def add_item(self, product_name, quantity, stock_available):
        """Add item to order with stock validation"""
        if quantity > stock_available:
            raise OutOfStockError(
                "Insufficient stock",
                order_id=self.order_id,
                product_name=product_name,
                requested_qty=quantity,
                available_qty=stock_available
            )
        self.items[product_name] = quantity
        return f"Added {quantity} x {product_name}"
    
    def apply_discount(self, discount_code):
        """Apply discount code"""
        valid_codes = ['SAVE10', 'SUMMER20']
        
        if discount_code not in valid_codes:
            raise InvalidDiscountCodeError(
                "Invalid discount code",
                order_id=self.order_id,
                discount_code=discount_code,
                reason="Code does not exist or has expired"
            )
        return f"Applied discount code: {discount_code}"
    
    def set_shipping_address(self, address):
        """Set shipping address with validation"""
        required_fields = ['street', 'city', 'zip', 'country']
        
        for field in required_fields:
            if field not in address or not address[field]:
                raise ShippingError(
                    f"Missing or empty field: {field}",
                    order_id=self.order_id,
                    address_field=field
                )
        return "Shipping address set successfully"

# Test e-commerce system
print("Creating order...")
order = Order("ORD-12345")

# Test 1: Add items
print("\nTest 1: Adding items")
try:
    print(order.add_item("Laptop", 1, 5))
    print(order.add_item("Mouse", 10, 3))  # Out of stock
except OutOfStockError as e:
    print(f"  Error: {e}")
    print(f"  Order ID: {e.order_id}")
    print(f"  Timestamp: {e.timestamp}")

# Test 2: Apply discount
print("\nTest 2: Applying discount")
try:
    print(order.apply_discount("SAVE10"))
    print(order.apply_discount("INVALID"))  # Invalid code
except InvalidDiscountCodeError as e:
    print(f"  Error: {e.args[0]}")
    print(f"  Code: {e.discount_code}")
    print(f"  Reason: {e.reason}")

# Test 3: Set shipping address
print("\nTest 3: Setting shipping address")
try:
    # Missing 'zip' field
    address = {'street': '123 Main St', 'city': 'New York', 'country': 'USA'}
    print(order.set_shipping_address(address))
except ShippingError as e:
    print(f"  Error: {e.args[0]}")
    print(f"  Order ID: {e.order_id}")
    print(f"  Missing field: {e.address_field}")

---

## 9. Advanced Custom Exceptions <a id='advanced-exceptions'></a>

Advanced techniques for custom exceptions.

In [ ]:
# Example: Exception with Context Manager Support

class TransactionError(Exception):
    """Exception for transaction errors with rollback support"""
    
    def __init__(self, message, transaction_id, operations_completed):
        super().__init__(message)
        self.transaction_id = transaction_id
        self.operations_completed = operations_completed
    
    def rollback(self):
        """Rollback completed operations"""
        print(f"Rolling back {len(self.operations_completed)} operations for transaction {self.transaction_id}")
        for op in reversed(self.operations_completed):
            print(f"  Undoing: {op}")

class Transaction:
    """Transaction context manager"""
    
    def __init__(self, transaction_id):
        self.transaction_id = transaction_id
        self.operations = []
    
    def __enter__(self):
        print(f"Starting transaction {self.transaction_id}")
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_type is None:
            print(f"Transaction {self.transaction_id} completed successfully")
        elif isinstance(exc_val, TransactionError):
            print(f"Transaction {self.transaction_id} failed: {exc_val}")
            exc_val.rollback()
            return True  # Suppress exception after rollback
        return False
    
    def execute(self, operation, fail=False):
        """Execute an operation"""
        if fail:
            raise TransactionError(
                f"Operation '{operation}' failed",
                transaction_id=self.transaction_id,
                operations_completed=self.operations.copy()
            )
        self.operations.append(operation)
        print(f"  Executed: {operation}")

# Test transaction with rollback
print("Test: Transaction with failure and rollback\n")
with Transaction("TXN-001") as txn:
    txn.execute("Debit account A")
    txn.execute("Credit account B")
    txn.execute("Update balance", fail=True)  # This will fail

---

## 10. When to Create Custom Exceptions <a id='when-to-create'></a>

### Create Custom Exceptions When:

1. **Domain-Specific Errors**: Your application has unique error conditions
   - Example: `PaymentDeclinedError`, `OutOfStockError`

2. **Need Additional Context**: Built-in exceptions don't capture enough information
   - Example: Storing error codes, transaction IDs, timestamps

3. **Hierarchical Error Handling**: Need to catch groups of related errors
   - Example: All database errors, all API errors

4. **API Design**: Building a library or framework
   - Users need specific exception types to handle

### Use Built-in Exceptions When:

1. **Common Error Types**: Error fits a built-in category
   - `ValueError` for invalid values
   - `TypeError` for type mismatches
   - `KeyError` for missing dictionary keys

2. **Simplicity**: No need for additional context

3. **Standard Behavior**: Exception behavior matches built-in type

In [ ]:
# Example: Deciding When to Create Custom Exceptions

# GOOD: Custom exception adds value
class EmailSendError(Exception):
    """Raised when email fails to send"""
    
    def __init__(self, message, recipient, smtp_code=None):
        super().__init__(message)
        self.recipient = recipient
        self.smtp_code = smtp_code

# GOOD: Use built-in exception (no additional context needed)
def divide(a, b):
    if b == 0:
        raise ZeroDivisionError("Cannot divide by zero")
    return a / b

# GOOD: Use built-in ValueError (standard validation)
def set_age(age):
    if age < 0:
        raise ValueError("Age cannot be negative")
    return age

# BAD: Unnecessary custom exception
# Don't do this when ValueError works fine:
class InvalidAgeException(Exception):
    pass

def bad_set_age(age):
    if age < 0:
        raise InvalidAgeException("Age cannot be negative")
    return age

print("Examples:")
print("1. Custom exception with context: EmailSendError")
print("2. Built-in exception works: ZeroDivisionError")
print("3. Built-in exception works: ValueError")
print("4. Unnecessary custom exception: InvalidAgeException (should use ValueError)")

---

## 11. Testing Custom Exceptions <a id='testing-exceptions'></a>

Testing is crucial for custom exceptions.

In [ ]:
# Example: Testing Custom Exceptions

class InsufficientBalanceError(Exception):
    """Raised when account balance is insufficient"""
    
    def __init__(self, message, balance, withdrawal_amount):
        super().__init__(message)
        self.balance = balance
        self.withdrawal_amount = withdrawal_amount

class Account:
    def __init__(self, balance):
        self.balance = balance
    
    def withdraw(self, amount):
        if amount > self.balance:
            raise InsufficientBalanceError(
                "Cannot withdraw more than available balance",
                balance=self.balance,
                withdrawal_amount=amount
            )
        self.balance -= amount
        return self.balance

# Test custom exception
def test_insufficient_balance_error():
    """Test that InsufficientBalanceError is raised correctly"""
    account = Account(100)
    
    # Test that exception is raised
    try:
        account.withdraw(150)
        print("FAIL: Exception was not raised")
    except InsufficientBalanceError as e:
        print("PASS: InsufficientBalanceError was raised")
        
        # Test exception message
        if "Cannot withdraw" in str(e):
            print("PASS: Exception message is correct")
        else:
            print("FAIL: Exception message is incorrect")
        
        # Test exception attributes
        if e.balance == 100:
            print("PASS: Exception balance attribute is correct")
        else:
            print("FAIL: Exception balance attribute is incorrect")
        
        if e.withdrawal_amount == 150:
            print("PASS: Exception withdrawal_amount attribute is correct")
        else:
            print("FAIL: Exception withdrawal_amount attribute is incorrect")
    
    # Test that valid withdrawal works
    try:
        new_balance = account.withdraw(50)
        if new_balance == 50:
            print("PASS: Valid withdrawal works correctly")
        else:
            print("FAIL: Valid withdrawal failed")
    except Exception as e:
        print(f"FAIL: Unexpected exception: {e}")

# Run tests
print("Running tests for InsufficientBalanceError:\n")
test_insufficient_balance_error()

---

## 12. Summary <a id='summary'></a>

### Key Takeaways:

1. **Custom Exceptions** extend Python's exception hierarchy for application-specific errors

2. **Creating Custom Exceptions**:
   ```python
   class MyError(Exception):
       pass
   ```

3. **Inheritance**: Always inherit from `Exception` or its subclasses

4. **Additional Data**: Store context information as attributes
   - Error codes
   - Timestamps
   - Field names
   - Suggested fixes

5. **Exception Chaining**: Preserve original exceptions with `from`
   ```python
   raise NewError("message") from original_error
   ```

6. **Exception Hierarchies**: Group related exceptions under base classes
   ```python
   class PaymentError(Exception): pass
   class PaymentDeclinedError(PaymentError): pass
   class InsufficientFundsError(PaymentDeclinedError): pass
   ```

### Best Practices:

1. Use descriptive names ending in "Error" or "Exception"
2. Add comprehensive docstrings
3. Include relevant context in attributes
4. Provide helpful error messages
5. Use exception chaining for better debugging
6. Create hierarchies for flexible error handling
7. Don't overuse custom exceptions
8. Test exception behavior thoroughly

### When to Create Custom Exceptions:

**Create Custom Exceptions When:**
- Domain-specific errors need specific handling
- Need to add additional context
- Building hierarchical error handling
- Designing public APIs

**Use Built-in Exceptions When:**
- Error fits a standard category
- No additional context needed
- Keeping it simple

### Common Patterns:

```python
# Basic custom exception
class CustomError(Exception):
    pass

# Exception with additional data
class DataError(Exception):
    def __init__(self, message, field_name, invalid_value):
        super().__init__(message)
        self.field_name = field_name
        self.invalid_value = invalid_value

# Exception hierarchy
class BaseError(Exception):
    pass

class SpecificError(BaseError):
    pass
```

### Benefits:

1. **Clarity**: Self-documenting error types
2. **Specificity**: Precise error information
3. **Control**: Better error handling flow
4. **Maintainability**: Easier to debug and fix
5. **Flexibility**: Can catch by category or specificity

### Real-World Applications:

- **Web Applications**: HTTP errors, authentication errors, validation errors
- **Database Systems**: Connection errors, query errors, transaction errors
- **File Processing**: Format errors, size errors, permission errors
- **E-commerce**: Payment errors, inventory errors, shipping errors
- **API Development**: Rate limit errors, authentication errors, request errors

Custom exceptions make your code more robust, maintainable, and user-friendly!